In [ ]:
#%pip freeze > requirements.txt

In [ ]:
from pathlib import Path
import shutil

In [ ]:
import os
import mimetypes
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload


In [ ]:
class YouTubeUploader:
    """
    A class to upload videos to YouTube, including support for YouTube Shorts.
    
    Requires:
    - pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
    - YouTube Data API v3 credentials (client_secrets.json)
    """
    
    # OAuth 2.0 scopes for YouTube upload and playlist management
    SCOPES = [
        'https://www.googleapis.com/auth/youtube.upload',
        'https://www.googleapis.com/auth/youtube'
    ]
    
    def __init__(self, credentials_file='client_secrets.json', token_file='token.json'):
        """
        Initialize the YouTube uploader.
        
        Args:
            credentials_file (str): Path to OAuth2 credentials JSON file
            token_file (str): Path to store the access token
        """
        self.credentials_file = credentials_file
        self.token_file = token_file
        self.youtube = None
        self._authenticate()
    
    def _authenticate(self):
        """Authenticate with YouTube API using OAuth2."""
        creds = None
        
        # Load existing token if available
        if os.path.exists(self.token_file):
            creds = Credentials.from_authorized_user_file(self.token_file, self.SCOPES)
        
        # If there are no valid credentials, request authorization
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.credentials_file, self.SCOPES)
                creds = flow.run_local_server(port=0)
            
            # Save credentials for future use
            with open(self.token_file, 'w') as token:
                token.write(creds.to_json())
        
        # Build the YouTube API client
        self.youtube = build('youtube', 'v3', credentials=creds)
        print("Successfully authenticated with YouTube API")
    
    def upload_video(self, video_path, title, description="", tags=None, 
                    category_id="22", privacy_status="private", is_short=False,
                    thumbnail_path=None, playlist_id=None):
        """
        Upload a video to YouTube.
        
        Args:
            video_path (str): Path to the video file
            title (str): Video title
            description (str): Video description
            tags (list): List of tags for the video
            category_id (str): YouTube category ID (22 = People & Blogs)
            privacy_status (str): 'private', 'public', 'unlisted'
            is_short (bool): True if this is a YouTube Short
            thumbnail_path (str): Optional path to custom thumbnail image file
            playlist_id (str): Optional playlist ID to add video to
            
        Note:
            YouTube automatically generates thumbnail options from your video.
            You can select from these auto-generated thumbnails in YouTube Studio,
            or upload a custom thumbnail using thumbnail_path parameter.
            
        Returns:
            dict: Upload response with video ID and URL
        """
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file not found: {video_path}")
        
        # Validate video file
        mime_type, _ = mimetypes.guess_type(video_path)
        if not mime_type or not mime_type.startswith('video/'):
            raise ValueError("File must be a video file")
        
        # Prepare tags
        if tags is None:
            tags = []
        
        # Add #Shorts tag for YouTube Shorts
        if is_short:
            if "#Shorts" not in tags and "#shorts" not in tags:
                tags.append("#Shorts")
            # Ensure description mentions it's a Short
            if "#Shorts" not in description and "#shorts" not in description:
                description = f"{description}\n\n#Shorts" if description else "#Shorts"
        
        # Video metadata
        body = {
            'snippet': {
                'title': title,
                'description': description,
                'tags': tags,
                'categoryId': category_id
            },
            'status': {
                'privacyStatus': privacy_status,
                'selfDeclaredMadeForKids': False
            }
        }
        
        # Create media upload object
        media = MediaFileUpload(
            video_path,
            chunksize=-1,
            resumable=True,
            mimetype=mime_type
        )
        
        try:
            # Execute the upload
            print(f"Uploading video: {title}")
            print(f"File: {video_path}")
            print(f"Is Short: {is_short}")
            
            insert_request = self.youtube.videos().insert(
                part=','.join(body.keys()),
                body=body,
                media_body=media
            )
            
            response = self._resumable_upload(insert_request)
            
            video_id = response['id']
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            
            print(f"Upload successful!")
            print(f"Video ID: {video_id}")
            print(f"Video URL: {video_url}")
            
            # Handle custom thumbnail upload if provided
            thumbnail_uploaded = False
            if thumbnail_path and os.path.exists(thumbnail_path):
                thumbnail_uploaded = self.upload_thumbnail(video_id, thumbnail_path)
                if thumbnail_uploaded:
                    print("Custom thumbnail uploaded. You can also choose from YouTube's auto-generated thumbnails in YouTube Studio.")
            
            # Add video to playlist if specified
            playlist_added = False
            if playlist_id:
                playlist_added = self.add_video_to_playlist(video_id, playlist_id)
            
            return {
                'video_id': video_id,
                'video_url': video_url,
                'title': title,
                'privacy_status': privacy_status,
                'is_short': is_short,
                'thumbnail_uploaded': thumbnail_uploaded,
                'playlist_added': playlist_added,
                'playlist_id': playlist_id if playlist_added else None,
                'youtube_studio_url': f"https://studio.youtube.com/video/{video_id}/edit"
            }
            
        except HttpError as e:
            print(f"HTTP error occurred: {e}")
            raise
        except Exception as e:
            print(f"An error occurred: {e}")
            raise
    
    def _resumable_upload(self, insert_request):
        """Handle resumable upload with progress tracking."""
        response = None
        error = None
        retry = 0
        
        while response is None:
            try:
                print("Uploading file...")
                status, response = insert_request.next_chunk()
                if status:
                    print(f"Upload progress: {int(status.progress() * 100)}%")
            except HttpError as e:
                if e.resp.status in [500, 502, 503, 504]:
                    error = f"Server error: {e}"
                    retry += 1
                    if retry > 5:
                        raise e
                else:
                    raise e
            except Exception as e:
                error = f"Unexpected error: {e}"
                raise e
        
        return response
    
    def upload_thumbnail(self, video_id, thumbnail_path):
        """
        Upload a custom thumbnail for a video.
        
        Args:
            video_id (str): YouTube video ID
            thumbnail_path (str): Path to thumbnail image file
            
        Returns:
            bool: True if successful, False otherwise
        """
        if not os.path.exists(thumbnail_path):
            print(f"Thumbnail file not found: {thumbnail_path}")
            return False
        
        # Validate image file
        mime_type, _ = mimetypes.guess_type(thumbnail_path)
        if not mime_type or not mime_type.startswith('image/'):
            print(f"File must be an image file. Found: {mime_type}")
            return False
        
        # Check file size (max 2MB for YouTube thumbnails)
        file_size = os.path.getsize(thumbnail_path)
        max_size = 2 * 1024 * 1024  # 2MB in bytes
        if file_size > max_size:
            print(f"Thumbnail file too large: {file_size} bytes (max: {max_size} bytes)")
            return False
        
        try:
            print(f"Uploading thumbnail for video {video_id}...")
            self.youtube.thumbnails().set(
                videoId=video_id,
                media_body=MediaFileUpload(thumbnail_path)
            ).execute()
            print(f"Thumbnail uploaded successfully for video {video_id}")
            return True
        except HttpError as e:
            print(f"Failed to upload thumbnail: {e}")
            return False
        except Exception as e:
            print(f"Unexpected error uploading thumbnail: {e}")
            return False
    
    def upload_short(self, video_path, title, description="", tags=None, 
                    privacy_status="private", thumbnail_path=None, playlist_id=None):
        """
        Convenience method specifically for uploading YouTube Shorts.
        
        Args:
            video_path (str): Path to the video file (should be vertical, ≤60 seconds)
            title (str): Video title
            description (str): Video description
            tags (list): List of tags
            privacy_status (str): Privacy setting
            thumbnail_path (str): Optional custom thumbnail path
            playlist_id (str): Optional playlist ID to add Short to
            
        Note:
            YouTube automatically generates thumbnail options from your Short.
            You can select from these in YouTube Studio or upload a custom one.
            
        Returns:
            dict: Upload response
        """
        return self.upload_video(
            video_path=video_path,
            title=title,
            description=description,
            tags=tags,
            privacy_status=privacy_status,
            is_short=True,
            thumbnail_path=thumbnail_path,
            playlist_id=playlist_id
        )
    
    def add_video_to_playlist(self, video_id, playlist_id):
        """
        Add a video to an existing playlist.
        
        Args:
            video_id (str): YouTube video ID
            playlist_id (str): YouTube playlist ID
            
        Returns:
            bool: True if successful, False otherwise
        """
        try:
            self.youtube.playlistItems().insert(
                part="snippet",
                body={
                    "snippet": {
                        "playlistId": playlist_id,
                        "resourceId": {
                            "kind": "youtube#video",
                            "videoId": video_id
                        }
                    }
                }
            ).execute()
            print(f"Video {video_id} added to playlist {playlist_id}")
            return True
        except HttpError as e:
            print(f"Failed to add video to playlist: {e}")
            return False
        except Exception as e:
            print(f"Unexpected error adding video to playlist: {e}")
            return False
    
    def create_playlist(self, title, description="", privacy_status="private"):
        """
        Create a new playlist.
        
        Args:
            title (str): Playlist title
            description (str): Playlist description
            privacy_status (str): 'private', 'public', 'unlisted'
            
        Returns:
            dict: Playlist information with ID, or None if failed
        """
        try:
            request = self.youtube.playlists().insert(
                part="snippet,status",
                body={
                    "snippet": {
                        "title": title,
                        "description": description
                    },
                    "status": {
                        "privacyStatus": privacy_status
                    }
                }
            )
            response = request.execute()
            
            playlist_id = response['id']
            playlist_url = f"https://www.youtube.com/playlist?list={playlist_id}"
            
            print(f"Playlist created: {title}")
            print(f"Playlist ID: {playlist_id}")
            print(f"Playlist URL: {playlist_url}")
            
            return {
                'playlist_id': playlist_id,
                'playlist_url': playlist_url,
                'title': title,
                'privacy_status': privacy_status
            }
            
        except HttpError as e:
            print(f"Failed to create playlist: {e}")
            return None
        except Exception as e:
            print(f"Unexpected error creating playlist: {e}")
            return None
    
    def get_playlists(self, max_results=25):
        """
        Get a list of the user's playlists.
        
        Args:
            max_results (int): Maximum number of playlists to return
            
        Returns:
            list: List of playlist information
        """
        try:
            request = self.youtube.playlists().list(
                part="snippet,contentDetails",
                mine=True,
                maxResults=max_results
            )
            response = request.execute()
            
            playlists = []
            for item in response['items']:
                playlists.append({
                    'id': item['id'],
                    'title': item['snippet']['title'],
                    'description': item['snippet']['description'],
                    'video_count': item['contentDetails']['itemCount'],
                    'url': f"https://www.youtube.com/playlist?list={item['id']}"
                })
            
            return playlists
            
        except HttpError as e:
            print(f"Failed to get playlists: {e}")
            return []
        except Exception as e:
            print(f"Unexpected error getting playlists: {e}")
            return []
    
    def upload_video_to_new_playlist(self, video_path, video_title, playlist_title, 
                                   video_description="", playlist_description="", 
                                   tags=None, privacy_status="private", is_short=False):
        """
        Upload a video and create a new playlist for it.
        
        Args:
            video_path (str): Path to the video file
            video_title (str): Video title
            playlist_title (str): Playlist title
            video_description (str): Video description
            playlist_description (str): Playlist description
            tags (list): Video tags
            privacy_status (str): Privacy setting for both video and playlist
            is_short (bool): True if this is a YouTube Short
            
        Returns:
            dict: Combined response with video and playlist information
        """
        # First create the playlist
        playlist_info = self.create_playlist(
            title=playlist_title,
            description=playlist_description,
            privacy_status=privacy_status
        )
        
        if not playlist_info:
            print("Failed to create playlist. Uploading video without playlist.")
            return self.upload_video(
                video_path=video_path,
                title=video_title,
                description=video_description,
                tags=tags,
                privacy_status=privacy_status,
                is_short=is_short
            )
        
        # Upload video to the new playlist
        video_result = self.upload_video(
            video_path=video_path,
            title=video_title,
            description=video_description,
            tags=tags,
            privacy_status=privacy_status,
            is_short=is_short,
            playlist_id=playlist_info['playlist_id']
        )
        
        # Combine results
        video_result['playlist_info'] = playlist_info
        return video_result
    def get_video_info(self, video_id):
        """
        Get information about an uploaded video.
        
        Args:
            video_id (str): YouTube video ID
            
        Returns:
            dict: Video information or None if not found
        """
        try:
            response = self.youtube.videos().list(
                part='snippet,statistics,status',
                id=video_id
            ).execute()
            
            if response['items']:
                return response['items'][0]
            else:
                return None
        except HttpError as e:
            print(f"Error retrieving video info: {e}")
            return None


In [ ]:

def get_playlistID(video_path):
    """
    Extracts the playlist ID from the video file name.
    
    Args:
        video_path (str): Path to the video file.
        
    Returns:
        str: Playlist ID corresponding to the video file name.
    """
    playListsDic={"Place":"PLqmee5eFM2mcU7j0a5YW-Ml9qHqJPp9wI",
              "Celeb":"PLqmee5eFM2mfMnIIL4Sz2-bTsj_FYPxqh",
              "Animal":"PLqmee5eFM2mdznFtxt9rZfWZJMHqOMQ2x",
              "Building":"PLqmee5eFM2mfJOAHC3b8IkqIrrXUeEBZU",
              "Object":"PLqmee5eFM2meCP7PnEGf4QOqcaapZPIs1"}


    file_name = Path(video_path).stem
    playlist_name = file_name[20:]  # Assuming the first 20 characters are fixed
    return playListsDic.get(playlist_name, None).strip()




In [ ]:
def get_first_file_with_extension(folder_path, extension):
    """
    Finds and returns the full path of the first file found in a given folder
    that matches the specified extension.

    Args:
        folder_path (str): The path to the folder to search.
        extension (str): The desired file extension (e.g., ".txt", ".csv").

    Returns:
        str or None: The full path of the first matching file, or None if no
                     file with the specified extension is found.
    """
    for filename in os.listdir(folder_path):
        if filename.endswith(extension):
            return os.path.join(folder_path, filename)
    return None


In [ ]:
cwd=Path(os.getcwd())

In [ ]:
video_path=cwd/"videos"/"2025-07-25-09-34-31_cities.mp4"
title="Can You Beat the Clock and Crack the Puzzle? ⏳🧩, #puzzle',#puzzles,#brainteaser"
tag_general=["AiArtStudio.Ai", '#shorts','#youtube shorts','#viral','#trending' ]
tag_puzzel=['#puzzle','#puzzles','#brainteaser','#riddle','#riddles','#puzzlechallenge','#mindpuzzle','#logicpuzzle','#puzzlelover',
                '#puzzleaddict','#solvepuzzle','#guesstheanswer','#puzzlegame']
tag_eng=['#can you solve it','#challengeyourself','#testyourbrain','#commentyouranswer','#letmeknow']
tags_all= tag_general + tag_puzzel + tag_eng
#title_tags=title+" " + " ".join(tag_eng)
description="Can you beat the clock and crack the puzzle? ⏳🧩 Join us for a thrilling challenge where you have just 40 seconds to solve a mind-bending puzzle! Test your skills, share the second that you solved the puzzel, and see if you can outsmart the clock! Don't forget to like, subscribe, and hit the notification bell for more exciting puzzles! "+" ".join(tags_all)
thumbnail=cwd/"assets"/"aiartstudio_logo.jpg"
CLIENTSECRETS=cwd/"assets"/"client_secret.json"
uploadedFolder=cwd/"videos"/"uploaded"
#print(title_tags)

In [ ]:
def uploadVideo(numberToUpload):
    """
    Uploads a video to YouTube as a Short with a custom thumbnail.
    
    Args:
        numberToUpload (int): The number of the video to upload.
        
    Returns:
        None
    """
    
    # Here you can add logic to select different videos based on the number
    # For now, we will use the same video_path and title for simplicity


    # Initialize uploader (make sure you have client_secrets.json)
    uploader = YouTubeUploader(credentials_file=CLIENTSECRETS)

    for i in range(numberToUpload):
        print(f"Uploading video number: {i}")
        video_path = get_first_file_with_extension(cwd/"videos", ".mp4")
        if not video_path:
            print("No video file found to upload.")
            break
        
        # Get playlist ID based on the video file name
        playListID = get_playlistID(video_path)
        if not playListID:
            print(f"No playlist ID found for video: {video_path}")
        
        
        print(f"Playlist ID for this video: {playListID}")
        # Upload Short with custom thumbnail image
        result = uploader.upload_short(
            video_path=video_path,
            title=title,
            description=description,
            tags=tags_all,
            privacy_status="private",
            thumbnail_path=thumbnail,
            playlist_id=playListID
        )
        if result:
            shutil.move(video_path, uploadedFolder)
            print(f"Video uploaded successfully: {result['video_url']}")
            print(f"Video ID: {result['video_id']}")
            print(f"Video URL: {result['video_url']}")
            print(f"Title: {result['title']}")
            print(f"Privacy Status: {result['privacy_status']}")
            print(f"Is Short: {result['is_short']}")
            print(f"Thumbnail Uploaded: {result['thumbnail_uploaded']}")
            print(f"YouTube Studio URL: {result['youtube_studio_url']}")
            print("Video moved to uploaded folder. ")

  

In [ ]:
uploadVideo(5)